In [35]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [6]:
url = "https://www.espn.com/nba/stats/player/_/table/offensive/sort/avgPoints/dir/desc"

# page = requests.get(url)
# soup = BeautifulSoup(page.text, "html.parser")

In [8]:
path = "../../Code/chromedriver"
driver = webdriver.Chrome(executable_path = path)
driver.get(url)

while True:
    try:
        showmore_button = driver.find_element_by_link_text('Show More')
        showmore_button.click()
    except:
        break

In [9]:
soup = BeautifulSoup(driver.page_source, "html.parser")

# driver.quit()

In [30]:
header = soup.find_all("tr", attrs = {"class": "Table__sub-header Table__TR Table__even"})[1]

NT_cols = []
NT_cols.append('NAME')
NT_cols.append('TEAM')

df_NT = pd.DataFrame(columns=NT_cols)

stats_cols = []
for col in header.find_all("th"):
    stats_cols.append(col.get_text())

df_columns = pd.DataFrame(columns=stats_cols)

# final_df = pd.concat([df_NT, df_columns], axis=1)
# final_df

,POS,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,...,FTA,FT%,REB,AST,STL,BLK,TO,DD2,TD3,PER


In [31]:
players = soup.find_all("tr", attrs = {"class": "Table__TR Table__TR--sm Table__even"})
# players
num = 0
for player in players:
    stats = [stat.get_text() for stat in player.find_all('td')]
    if len(stats) == 2:
        string = player.get_text()
        i = -1
        while(string[i].isupper() or string[i]=="/"):
            if(i <= -3):
                if(string[i] == "I" and string[i+1] == "I" and string[i+2] == "I"):
                    i += 2
                    break
            i -= 1
        data = []
        data.append(string[:i+1])
        data.append(string[i+1:])
        temp_df = pd.DataFrame(data).transpose()
        temp_df.columns = NT_cols
        df_NT = pd.concat([df_NT, temp_df], ignore_index=True)
#         print(name)
#         print(num)
        num += 1
    else:
        stats = [stat.get_text() for stat in player.find_all('td')]
        
        temp_df = pd.DataFrame(stats).transpose()
        temp_df.columns = stats_cols
        
        df_columns = pd.concat([df_columns, temp_df], ignore_index=True)

In [32]:
# col = soup.find("tr", attrs = {"class": "Table2__tr Table2__tr--sm Table2__even"})
# # col
# for data in col.find_all("td"):
#     string = data.get_text()
#     i = -1
#     while(string[i].isupper() or string[i]=="/"):
#         i -= 1
#     name = string[:i+1]
#     team = string[i+1:]
#     print(name)

In [33]:
final_df = pd.concat([df_NT, df_columns], axis=1)
final_df

,NAME,TEAM,POS,GP,MIN,PTS,FGM,FGA,FG%,3PM,...,FTA,FT%,REB,AST,STL,BLK,TO,DD2,TD3,PER
0,1Bradley Beal,WSH,SG,10,35.9,34.9,12.1,24.5,49.4,2.4,...,9.5,87.4,5.3,5.0,1.4,0.7,3.3,1,0,27.43
1,2Kevin Durant,BKN,PF,12,36.3,31.3,10.4,19.5,53.4,2.8,...,8.8,86.7,7.5,6.0,0.8,1.3,3.7,5,0,28.32
2,3Stephen Curry,GS,PG,15,34.3,28.2,9.1,20.3,45.1,4.1,...,6.2,93.5,5.5,6.1,1.3,0.1,3.5,4,0,22.82
3,4Damian Lillard,POR,PG,14,36.2,28.1,8.5,19.3,44.1,3.7,...,7.9,94.5,4.7,6.7,1.0,0.3,3.1,1,0,24.42
4,5Joel Embiid,PHI,C,13,31.3,27.3,8.7,15.5,55.9,1.2,...,10.2,85.0,11.3,2.8,1.3,1.3,3.2,10,0,30.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,261Harry Giles III,POR,PF,10,9.0,2.5,0.9,2.1,42.9,0.2,...,0.8,62.5,3.3,0.6,0.4,0.5,0.6,0,0,11.69
261,262Dean Wade,CLE,PF,13,10.7,2.4,0.8,2.3,36.7,0.5,...,0.2,100.0,1.4,0.5,0.4,0.1,0.5,0,0,6.39
262,263Ed Davis,MIN,F,11,13.6,2.3,0.9,2.5,37.0,0.0,...,0.5,83.3,5.2,0.9,0.5,0.5,0.3,0,0,12.69
263,264Josh Green,DAL,SG,13,13.5,2.2,0.9,2.2,42.9,0.0,...,0.6,62.5,2.8,0.7,0.2,0.0,0.6,0,0,5.57


In [34]:
final_df.to_csv("../Data/stats18-19.csv", index=False, sep=',', encoding='utf-8')